In [8]:
%matplotlib inline
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

#load the data set + convert it to type numpy compatible with tensorflow
url = 'https://raw.githubusercontent.com/MyaraB/Deep-Learning-and-NLP/main/dataset_full.csv'
df1 = pd.read_csv(url)
df = pd.DataFrame(df1)
df = df.astype('float32')
df=df.to_numpy()

#shuffle data
np.random.shuffle(df)

#manually split the data into 4sets + 1 validation set
training_set1 = df[0:14080]
test_set1 = df[14080:17600]
training_set2 = df[17600:31680]
test_set2 = df[31680:35200]
training_set3 = df[35200:49280]
test_set3 = df[49280:52800]
training_set4 = df[52800:66880]
test_set4 = df[66880:70400]
validation_set= df[70400:] 

#manually choosing 14 relevant features from the 111 available to us in the dataset
#features used: qty_dot_url (1), qty_hyphen_url(2), qty_underline_url(3)
#,qty_slash_url(4),qty_equal_url(6), qty_at_url(7),qty_and_url(8), 
#qty_percent_url(17),qty_tld_url(18), length_url(19),qty_dot_domain(20),
#qty_hyphen_domain(21), qty_vowels_domain(37), domain_length(38),phishing(111)

training_set1_features = np.array(training_set1[:,[0,1,2,3,5,6,7,16,17,18,19,20,36,37]])
training_set1_labels = np.array(training_set1[:,-1])
test_set1_features = np.array(test_set1[:,[0,1,2,3,5,6,7,16,17,18,19,20,36,37]])
test_set1_labels = np.array(test_set1[:,-1])

features = 14
eps = 1e-12
learning_rates = 0.002

#shaping the data in-order for it to be usable with tensor
data_x = training_set1_features
data_y = training_set1_labels.reshape((-1,1))
test_set1_labels = test_set1_labels.reshape((-1,1))

x = tf.placeholder(tf.float32,[None,features])
y_ = tf.placeholder(tf.float32,[None,1])
W = tf.Variable(tf.zeros([features,1]))
b = tf.Variable(tf.zeros([1]))
y = tf.nn.sigmoid(tf.matmul(x,W)+b)

loss=tf.reduce_mean(-(y_*tf.log(y + eps) + (1 - y_+eps)*tf.log(1 - y+eps)))

update=tf.train.GradientDescentOptimizer(learning_rates).minimize(loss)

prediction = tf.round(tf.sigmoid(tf.matmul(x,W)+b))
correct = tf.cast(tf.equal(prediction, y_), dtype=tf.float32) 
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(0,14000):
    sess.run(update, feed_dict = {x:data_x, y_:data_y}) #BGD 

    if (epoch+1)%1000==0: #print every 1000 iterations
        err, _ = sess.run([loss, update], {x: data_x, y_:data_y})
        feeds_train = {x:data_x, y_:data_y}
        feeds_test = {x:test_set1_features, y_:test_set1_labels}
        train_acc = sess.run(accuracy, feed_dict=feeds_train)
        test_acc = sess.run(accuracy, feed_dict=feeds_test)
        print ("epoch: %3d train accuracy: %.3f test accuracy: %.3f loss: %.3f" % (epoch+1,train_acc, test_acc, err))
       

epoch: 1000 train accuracy: 0.861 test accuracy: 0.868 loss: 0.382
epoch: 2000 train accuracy: 0.861 test accuracy: 0.867 loss: 0.356
epoch: 3000 train accuracy: 0.869 test accuracy: 0.876 loss: 0.340
epoch: 4000 train accuracy: 0.872 test accuracy: 0.879 loss: 0.330
epoch: 5000 train accuracy: 0.874 test accuracy: 0.879 loss: 0.323
epoch: 6000 train accuracy: 0.875 test accuracy: 0.882 loss: 0.317
epoch: 7000 train accuracy: 0.875 test accuracy: 0.883 loss: 0.312
epoch: 8000 train accuracy: 0.875 test accuracy: 0.882 loss: 0.308
epoch: 9000 train accuracy: 0.876 test accuracy: 0.882 loss: 0.305
epoch: 10000 train accuracy: 0.876 test accuracy: 0.881 loss: 0.302
epoch: 11000 train accuracy: 0.877 test accuracy: 0.882 loss: 0.300
epoch: 12000 train accuracy: 0.878 test accuracy: 0.882 loss: 0.297
epoch: 13000 train accuracy: 0.879 test accuracy: 0.883 loss: 0.295
epoch: 14000 train accuracy: 0.879 test accuracy: 0.883 loss: 0.294
